Коронавирус застал мир врасплох, изменив привычный порядок вещей. 

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

<b> Постановка задачи </b>

Наша задача — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Это нужно для формулирования ценностного предложения продукта.

<b> Нужно: </b>
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Импорт библиотек и данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from termcolor import colored

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
# Функция вызова
def select(sql):
    return pd.io.sql.read_sql(sql, con = engine)

In [4]:
query_1 = '''
SELECT * FROM books
'''
books = pd.io.sql.read_sql(query_1, con = engine)

query_2 = '''
SELECT * FROM authors
'''
authors = pd.io.sql.read_sql(query_2, con = engine)

query_3 = '''
SELECT * FROM publishers
'''
publishers = pd.io.sql.read_sql(query_3, con = engine)

query_4 = '''
SELECT * FROM ratings
'''

ratings = pd.io.sql.read_sql(query_4, con = engine)

query_5 = '''
SELECT * FROM reviews
'''
reviews = pd.io.sql.read_sql(query_5, con = engine)

## Таблицы

Выведем первые строки полученных таблиц

In [5]:
# Моя собственная функция для первичного анализа загружаемых таблиц
# Чтобы сильно не удлинять, закоментил некоторые выводы.

def base_info(df):
    
#    df.columns = [col.lower().replace(' ','_').replace('-','_') for col in df.columns]
    
    print(colored("Общая информация о таблице:", "blue", attrs=['bold']))
    print()
    df.info()
    print('*'*70)
#    print(colored("Циферки:", "blue", attrs=['bold']))
#    display(df.describe().T)
#    print('*'*70)
    print(colored("Количество явных дубликатов: ", "blue", attrs=['bold']), df.duplicated().sum())
    print('*'*70)
    print(colored("Доли отсутствующих значений", "blue", attrs=['bold']))
    print(round(df.isna().mean() * 100, 2))
    print('*'*70)
#    if len(df.index) > 10:
#            print(colored('Случайная выборка 10 строк таблицы:', "blue", attrs=['bold']))
#            display(df.sample(n=10, random_state=10))
#    else:
#            print('Первые 10 строк таблицы:')
#            display(df.head(10))

In [6]:
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Предварительный анализ
Посмотрим на загруженные таблице более детально:

In [11]:
base_info(books)

Общая информация о таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
**********************************************************************
Количество явных дубликатов:  0
**********************************************************************
Доли отсутствующих значений
book_id             0.0
author_id           0.0
title               0.0
num_pages           0.0
publication_date    0.0
publisher_id        0.0
dtype: float64
**********************************************************************


In [12]:
base_info(authors)

Общая информация о таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
**********************************************************************
Количество явных дубликатов:  0
**********************************************************************
Доли отсутствующих значений
author_id    0.0
author       0.0
dtype: float64
**********************************************************************


In [13]:
base_info(publishers)

Общая информация о таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
**********************************************************************
Количество явных дубликатов:  0
**********************************************************************
Доли отсутствующих значений
publisher_id    0.0
publisher       0.0
dtype: float64
**********************************************************************


In [14]:
base_info(ratings)

Общая информация о таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
**********************************************************************
Количество явных дубликатов:  0
**********************************************************************
Доли отсутствующих значений
rating_id    0.0
book_id      0.0
username     0.0
rating       0.0
dtype: float64
**********************************************************************


In [15]:
ratings.book_id.nunique()

1000

In [16]:
base_info(reviews)

Общая информация о таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
**********************************************************************
Количество явных дубликатов:  0
**********************************************************************
Доли отсутствующих значений
review_id    0.0
book_id      0.0
username     0.0
text         0.0
dtype: float64
**********************************************************************


In [17]:
reviews.book_id.nunique()

994

### Вывод:

Итак у нас есть датасет с 1000 книг от 636 авторов, выпущенными 340 издателями. Что порадовало, так это отсутствие пропусков и явных дубликатов во всех таблицах. А вот рецензии есть не на все книги.

## Решение задач

### Количество книг, вышедших после 1 января 2000 года

In [18]:
books_count = '''
SELECT COUNT(book_id) AS books_count FROM books WHERE publication_date > '2000-01-01'
'''
select(books_count)

,books_count
0,819


ПОСЛЕ 1 января 2020 вышло 819 книг

### Количество обзоров и средняя оценка

In [19]:
ratings_rewies = '''
SELECT 
    books.title AS title,
    books.book_id AS id,
    AVG(ratings.rating) AS avg_rating,
    COUNT(DISTINCT reviews.text) AS count_reviews
FROM
    books
FULL OUTER JOIN ratings ON ratings.book_id = books.book_id
FULL OUTER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
     books.title,
     books.book_id
order by
     count_reviews DESC

'''
select(ratings_rewies)

,title,id,avg_rating,count_reviews
0,Twilight (Twilight #1),948,3.662500,7
1,Water for Elephants,963,3.977273,6
2,The Glass Castle,734,4.206897,6
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,4.414634,6
4,The Curious Incident of the Dog in the Night-Time,695,4.081081,6
...,...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,83,3.666667,0
996,The Natural Way to Draw,808,3.000000,0
997,The Cat in the Hat and Other Dr. Seuss Favorites,672,5.000000,0
998,Essential Tales and Poems,221,4.000000,0


Макчимальное количество обзоров у книги "Сумерки" - 7шт, а вот средний рейтиинг всего 3,66

### Издательство, которое издало наибольшее число книг 

In [20]:
book_publisher = '''
SELECT
     publishers.publisher AS name_publisher,
     COUNT(books.book_id) AS count_published_books
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50
GROUP BY
    name_publisher
ORDER BY 
    count_published_books DESC
'''
select(book_publisher)

,name_publisher,count_published_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


С отрывом идёт издательство "Penguin Books" с 42 изданными книгами, толще 50 страниц

### Автор с самой высокой средней оценкой книг

In [21]:
best_author = '''
SELECT
      SUBQ.author AS author,
      AVG(SUBQ.avg_rating) AS avg_rating
FROM
(SELECT
     authors.author AS author,
     books.book_id AS id,
     AVG(ratings.rating) AS avg_rating,
     COUNT(ratings.rating) AS count_rating
FROM
    authors
INNER JOIN books ON books.author_id = authors.author_id
INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    author,
    id
    HAVING
    COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
'''
select(best_author)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самым рейтинговым оказался тандем писательницы Роулинг и иллюстратора Гранпрэ книг о Гарри Поттере. Интересно, что автора указывают сразу с иллюстратором...

### Среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

In [22]:
avg_text = '''
SELECT ROUND(avg(count)) 
FROM
(SELECT
    COUNT(text)
FROM 
    reviews
FULL OUTER JOIN (SELECT username,
                COUNT(rating_id) as count_rate
FROM 
    ratings
GROUP BY
        username) AS count_rating
ON count_rating.username = reviews.username
where count_rate > 50
GROUP BY reviews.username) as count;                                  
'''
select(avg_text)

,round
0,24.0


В среднем люди, поставившие более 50 оценок написали по 24 текстовых обзора на книги.

## Общий вывод

В нашем проекте мы выяснили:
1. Сколько книг вышло после 1 января 2000 года - `819штук`; 
2. Для каждой книги посчитали количество обзоров и среднюю оценку; 
3. Определили издательство, которое выпустило наибольшее число книг толще 50 страниц — `Penguin Books` с 42 изданными книгами; 
4. Определили автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками - `J.K. Rowling` 
5. Узнали среднее количество обзоров от пользователей, которые поставили больше 50 оценок - `24`.

Важный момент: Мы можем получать не совсем верные рейтинги, скорее стоит опираться на обзоры и их количество, там где их больше рейтинг будет честнее.